In [1]:
# -- use this line at the beginning of your notebook to turn on interactive plots
%matplotlib notebook

# Imports
import sys
import matplotlib.pyplot as plt  # plotting library
from matplotlib import colors
import numpy as np  # work with numeric arrays without labeled axes
import xarray as xr  # work with arrays with labeled axes
import xrscipy.signal as dsp  # xarray signal filtering etc.
import scipy as sps
from cdb_extras import xarray_support as cdbxr  # access to COMPASS Database (CDB)
import pickle # to save data
from pathlib import Path # to easily work with different files
from progressbar import ProgressBar
from typing import Optional, Union
from dataclasses import dataclass
from typing import Optional
plt.rcParams.update({'font.size': 14})

/compass/home/odlozilik/.local/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
from core.pedestal_fit import PedestalParams, get_thomson_data, fit, pedestal_params_extraction

In [3]:
# Only use Thomson data from this normalized psi value and up
psi_n_lim = 0.6

In [4]:
help(pedestal_params_extraction)

Help on function pedestal_params_extraction in module core.pedestal_fit:

pedestal_params_extraction(shot_nr: int, psi_n_lim: int, save_path: Union[pathlib.Path, str, NoneType] = None)
    Takes one shot number, and fits the pedestal of all thomson scattering profiles recorded within H-modes. Also calculates 
    the position of the TS measurement in relation to the sawteeth and the ELMs, both in units of time and phase.
    
    From the fits, it calculates the pedestal height, width, and gradient for pressure, density and temperature, as well 
    as their fit errors in standard deviations.
    
    To automatically save the results, pass a folder path as `save_path`.



In [5]:
test=pedestal_params_extraction(18252, psi_n_lim)

Using stray corrected data


  0% (0 of 55) |                         | Elapsed Time: 0:00:00 ETA:  --:--:--

Number of H-mode intervals : 1


 27% (15 of 55) |######                  | Elapsed Time: 0:00:03 ETA:   0:00:24

Could not fit at 1105ms. Reason: Optimal parameters not found: Number of calls to function has reached maxfev = 1000.


...

 43% (24 of 55) |##########              | Elapsed Time: 0:00:18 ETA:   0:00:33

Huge error on b_height at 1180ms. err: 3.4e+02, val:3.4e+02. Dropping it.


 47% (26 of 55) |###########             | Elapsed Time: 0:00:20 ETA:   0:00:29

Could not fit 1 out of 55 measurements.


AttributeError: scipy.signal.spectral is deprecated and has no attribute _spectral_helper. Try looking in scipy.signal instead.